In [1]:
import os, sys, subprocess, venv, pathlib, platform, json, site

# Path: ./.qnn_hybrid_venv  (lives alongside the notebook)
venv_dir = pathlib.Path(".qnn_hybrid_venv").resolve()
if not venv_dir.exists():
    print("▶️  Creating venv:", venv_dir)
    venv.create(venv_dir, with_pip=True)
else:
    print("✅  Reusing venv:", venv_dir)

# Helpers for later cells
BIN = "Scripts" if platform.system() == "Windows" else "bin"
pip_cmd    = venv_dir / BIN / "pip"
python_cmd = venv_dir / BIN / "python"

print("pip  =", pip_cmd)
print("python =", python_cmd)

✅  Reusing venv: /Users/grantmckenzie/.qnn_hybrid_venv
pip  = /Users/grantmckenzie/.qnn_hybrid_venv/bin/pip
python = /Users/grantmckenzie/.qnn_hybrid_venv/bin/python


In [2]:
import subprocess, shlex

def run(cmd):
    print("🔧", " ".join(map(str, cmd)))
    subprocess.check_call(list(map(str, cmd)))

# 0) Basic tooling & ipykernel
run([pip_cmd, "install", "--upgrade", "pip", "wheel", "setuptools", "ipykernel"])

# 1) Keep Numpy ≥2 to satisfy modern Qiskit
run([pip_cmd, "install", "--upgrade", "numpy>=2,<3"])

# 2) Torch (CPU build shown; swap out the URL if you need CUDA)
run([pip_cmd, "install", "torch", "torchvision", "torchaudio",
     "--index-url", "https://download.pytorch.org/whl/cpu"])

# 3) SNN + helpers  (tonic installed *without* its outdated pins)
run([pip_cmd, "install", "snntorch", "norse", "mne", "scipy",
     "matplotlib", "pywavelets", "tqdm"])
run([pip_cmd, "install", "--no-deps", "tonic"])

# 4) Quantum tool-chain
run([pip_cmd, "install", "qiskit[visualization]", "qiskit-ibm-runtime",
     "qiskit-machine-learning>=0.8.3"])

🔧 /Users/grantmckenzie/.qnn_hybrid_venv/bin/pip install --upgrade pip wheel setuptools ipykernel
🔧 /Users/grantmckenzie/.qnn_hybrid_venv/bin/pip install --upgrade numpy>=2,<3
🔧 /Users/grantmckenzie/.qnn_hybrid_venv/bin/pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
🔧 /Users/grantmckenzie/.qnn_hybrid_venv/bin/pip install snntorch norse mne scipy matplotlib pywavelets tqdm
🔧 /Users/grantmckenzie/.qnn_hybrid_venv/bin/pip install --no-deps tonic
🔧 /Users/grantmckenzie/.qnn_hybrid_venv/bin/pip install qiskit[visualization] qiskit-ibm-runtime qiskit-machine-learning>=0.8.3


In [3]:
import subprocess, json

display_name = "Python (qnn-hybrid)"
kernel_name  = "qnn-hybrid"

print(f"🖨️  Registering kernel: {display_name}")
subprocess.check_call([
    python_cmd, "-m", "ipykernel", "install",
    "--user", "--name", kernel_name, "--display-name", display_name
])
print("✅  Done — now select it via  Kernel ▸ Change kernel ▸", display_name)

🖨️  Registering kernel: Python (qnn-hybrid)
Installed kernelspec qnn-hybrid in /Users/grantmckenzie/Library/Jupyter/kernels/qnn-hybrid
✅  Done — now select it via  Kernel ▸ Change kernel ▸ Python (qnn-hybrid)


In [4]:
import sys, qiskit_machine_learning as qml
print("Python exec :", sys.executable)
print("QML version :", qml.__version__)

Python exec : /Users/grantmckenzie/.qnn_hybrid_venv/bin/python
QML version : 0.8.3


In [5]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_machine_learning.algorithms import QSVC

# ↓ NEW import path ↓
try:
    from qiskit_machine_learning.kernels.quantum_kernel import QuantumKernel
except ImportError:                       # for future versions ≥0.9
    from qiskit_machine_learning.kernels import FidelityQuantumKernel as QuantumKernel

from qiskit.circuit.library import ZZFeatureMap
from qiskit import Aer
from sklearn.datasets import make_classification

# Run **ONCE**, then comment out – insert your real token
# QiskitRuntimeService.save_account("PASTE_YOUR_IBM_TOKEN_HERE", overwrite=True)

service = QiskitRuntimeService()
print("Simulators:", [b.backend_name for b in service.backends(simulator=True)][:3])

backend = Aer.get_backend("aer_simulator_statevector")
kernel  = QuantumKernel(feature_map=ZZFeatureMap(2), backend=backend)

X, y = make_classification(n_samples=20, n_features=2, random_state=0)
model = QSVC(quantum_kernel=kernel).fit(X, y)
print("Training accuracy:", model.score(X, y))

ImportError: cannot import name 'Aer' from 'qiskit' (/Users/grantmckenzie/.qnn_hybrid_venv/lib/python3.10/site-packages/qiskit/__init__.py)